In [3]:
%matplotlib inline
import os
import ftplib
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import cartopy.feature as cfeat
import cartopy.crs as ccrs
import seaborn.apionly as sns

In [2]:
# load paths
website = 'ftp-cdc.dwd.de'
webdir = '/pub/CDC/observations_germany/climate/daily/kl/recent/'
zipdir = 'F:\\Documenten\\Documenten\\CDC\\zip\\'
txtdir = 'F:\\Documenten\\Documenten\\CDC\\txt\\'

In [3]:
## download zipfiles
ftp = ftplib.FTP(website)
ftp.login()
ftp.cwd(webdir)

dirlist = []
filenames = ftp.nlst()

for filename in filenames:
    local_filename = os.path.join(zipdir, filename)
    file = open(local_filename, 'wb')
    ftp.retrbinary('RETR '+ filename, file.write)
    file.close()
    
ftp.quit()
print 'done downloading zips'

done downloading zips


In [4]:
# unpack zipfiles
i = 0
local_filenames = os.listdir(zipdir)
for filename in local_filenames: 
    if filename.endswith('.zip'): 
        fullname = os.path.join(zipdir, filename)
        fh = open(fullname, 'rb')
        z = zipfile.ZipFile(fh)
        name = z.namelist()[6]
        
        # make new folder
        if i == 0:
            foldername = '_'.join(name.split('_')[:5]) + '\\'
            folderdir = os.path.join(txtdir, foldername)
            try:
                os.mkdir(folderdir)
            except:
                pass
            
        i += 1
        filepath = os.path.join(folderdir, name)
        if not os.path.isfile(filepath):
            z.extract(name, folderdir)
        fh.close()
        
print 'done unpacking zips'

done unpacking zips


In [7]:
namesplit = name.split('_')
startdate = namesplit[3]
enddate = namesplit[4]

In [8]:
# read txt files into pandas panel
txtnames = [folderdir + txtname for txtname in os.listdir(folderdir)]

dflist = [pd.read_csv(txtname, delimiter = ';', engine = 'python', index_col = ' MESS_DATUM') for txtname in txtnames[:-2]]
df_concat = pd.concat(dflist, axis = 0)
df_concat.index = [df_concat.index,df_concat.STATIONS_ID]

data = df_concat.to_panel()
data = data.transpose(2,1,0)

# convert index to datetime
data.major_axis = pd.to_datetime(data.major_axis, format = '%Y%m%d')

# read metadata
metadata = pd.read_fwf(zipdir + 'KL_Tageswerte_Beschreibung_Stationen.txt', 
                       index_col = 'Stations_id', 
                       widths = [11, 10, 10, 15, 10, 10, 42, 92],
                       skiprows = [1],
                       encoding='ANSI')

metadata.columns = ['start','end','height','lat','lon','name','state']

# replace station id with names
#data.items = metadata['name'].loc[data.items.values]

# make subselection on variables
data = data.iloc[:,:,2:-1]

# assign new minor axis names
data.minor_axis = ['T','e','Octo','p','r','wind','Tx','Tn','Tn_10','wind_max','precip','precip_ind','sunhours','snowheight']

# set all -999 to NaN
#data = data[data != -999]

In [2]:
stations_var = data.loc[:,'2016/11/01':enddate,'T']
stations_var = stations_var[stations_var != -999]
stations_var = stations_var.dropna(axis = 1)

stations_var[stations_var > 0] = 0
stations_var = -1. * stations_var.sum()

stations_lat = metadata['lat'].loc[data.items.values]
stations_lon = metadata['lon'].loc[data.items.values]
#stations_name = metadata['name'].loc[data.items.values]

NameError: name 'data' is not defined

In [3]:
data2 = pd.concat([stations_lat,stations_lon,stations_var], axis = 1)
data2.columns = ['lat','lon','vari']
data2 = data2.dropna()
hellmann_int = data2.vari.astype(int)

NameError: name 'stations_lat' is not defined

In [4]:
cmap_rgb = sns.dark_palette('blue', 201)
cmap_ints = hellmann_int.copy()
cmap_ints[cmap_ints > 200] = 200

NameError: name 'hellmann_int' is not defined

In [5]:
vari = np.floor(data2.vari).astype(int)

NameError: name 'data2' is not defined

In [4]:
def _plot_natural_earth_feature(category, name, scale, fc, ec, lw, ls, a):
    feature = cfeat.NaturalEarthFeature(category = category, name = name, scale = scale, 
                                        facecolor = fc, edgecolor = ec, linewidth = lw, linestyle = ls, alpha = a)
    return ax.add_feature(feature)

In [ ]:
# set up map projection
proj = ccrs.Miller()
fig = plt.figure(figsize = (50,25))
ax = plt.axes(projection = proj)

# set edges of the map
extent_lon_min = 3#5.2
extent_lon_max = 15.2
extent_lat_min = 47.3
extent_lat_max = 55.1

extent = [extent_lon_min, extent_lon_max, extent_lat_min, extent_lat_max]
ax.set_extent(extent)

lw = 1.
a = 1.

scale = '10m'

# draw map background
_plot_natural_earth_feature('physical', 'ocean', scale, cfeat.COLORS['water'], 'none', lw, '-', a)
_plot_natural_earth_feature('physical', 'land', scale, cfeat.COLORS['land'], 'face', lw, '-', a)
_plot_natural_earth_feature('physical', 'lakes', scale, cfeat.COLORS['water'], 'black', lw, '-', a)
_plot_natural_earth_feature('cultural', 'admin_0_boundary_lines_land', scale, 'none', 'black', lw, '-', a)
_plot_natural_earth_feature('cultural', 'admin_1_states_provinces_lines', scale, 'none', 'black', 0.5 * lw, '--', a)
_plot_natural_earth_feature('cultural', 'roads', scale, 'none', 'red', 0.25*lw, '-', a)
_plot_natural_earth_feature('cultural', 'urban_areas', scale, 'orange', 'none', lw, '-', .25)

ax.coastlines(resolution = scale, linewidth = lw)


# plot data
#for lon, lat, text, cmap_int in zip(data2.lon, data2.lat, hellmann_int, cmap_ints):
#    ax.text(lon, lat, text, transform = ccrs.PlateCarree(), ha = 'center', va = 'center', color = cmap_rgb[cmap_int], alpha = .75, fontsize = 14,
#           path_effects=[PathEffects.withStroke(linewidth = 3, foreground="w")])

#scat = ax.scatter(lonlist, latlist, s = .001 * poplist, c = 'lightblue', transform = ccrs.PlateCarree(), linewidth = .5, alpha = .75, zorder = 2)
#plt.colorbar(scat)

#for lon, lat, text in zip(lonlist, latlist, namelist):
#    ax.text(lon, lat, text, transform = ccrs.PlateCarree(), ha = 'center', va = 'center', color = 'black', alpha = .75,
#           path_effects=[PathEffects.withStroke(linewidth = 3, foreground="w")])

# add title 
#text_title = 'Hellmann Scores of DWD Stations\n2016/11/01 to %s'%(enddate[:4] + '/' + enddate[4:6] + '/' + enddate[6:])
text_hellmann = 'Hellmann Score = $\Sigma (T_{g} < 0)$'
text_copyright = '(c) Lars Tijssen'


#ax.annotate(text_title, xy = (.01,.99), xycoords = 'axes fraction', fontsize = 24, va = 'top',
#            path_effects=[PathEffects.withStroke(linewidth = 4, foreground="w")])

#ax.annotate(text_hellmann, xy = (.01,.925), xycoords = 'axes fraction', fontsize = 24, va = 'top',
#            path_effects=[PathEffects.withStroke(linewidth = 4, foreground="w")])

#ax.annotate(text_copyright, xy = (.01,.01), xycoords = 'axes fraction', fontsize = 24, va = 'bottom',
#            path_effects=[PathEffects.withStroke(linewidth = 4, foreground="w")])

#plt.savefig('F:\\Documenten\\Documenten\\CDC\\Hellmann_20161101_%s.pdf'%enddate, bbox_inches = 'tight', transparent = True)
#plt.savefig('F:\\Documenten\\Documenten\\CDC\\Hellmann_20161101_%s.png'%enddate, bbox_inches = 'tight', transparent = True)